## Домашнее задание

1. Попробуйте видоизменить параметры разобранной на уроке двухслойной нейронной сети таким образом, чтобы улучшить ее точность (число нейронов, число эпох , можно изменять число слоев).
2. Проведите анализ — что приводит к ухудшению точности нейронной сети? Что приводит к увеличению ее точности?

In [1]:
'''
Исходный код к уроку 1.
Построение двухслойной нейронный сети для классификации цветков ириса
'''

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split

In [2]:
### Шаг 1. Определение функций, которые понадобяться для обучения
# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = Y[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return (x)*(1 - (x))

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [3]:

### Шаг 2. Подготовка тренировочных данных
from google.colab import drive
drive.mount('/content/drive')
# получения данных из csv файла. укажите здесь путь к файлу Iris.csv
with open('/content/drive/My Drive/Colab Notebooks/Iris.csv', 'r') as f:
    iris_data = pd.read_csv(f)

# замена текстовых значений на цифровые
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.values)

# формирование выходных данных(результатов)
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

Mounted at /content/drive


In [4]:
### Шаг 3. Обученние нейронной сети

accuracies = { }

# определим число нейронов скрытого слоя
neurons = [5, 10, 15, 25]

for neuron_numb in neurons:
    # присваевание случайных весов
    w0 = 2*np.random.random((4, neuron_numb)) - 1 # для входного слоя   - 4 входа, 3 выхода
    w1 = 2*np.random.random((neuron_numb, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

    # скорость обучения (learning rate)
    speed = [0.05, 0.1, 0.15]
    for n in speed:
        # массив для ошибок
        errors = []

        epoch = [3000, 4000, 5000]
        # процесс обучения
        for epo in epoch:
            for i in range(epo):

                # прямое распространение(feed forward)
                layer0 = X_train
                layer1 = sigmoid(np.dot(layer0, w0))
                layer2 = sigmoid(np.dot(layer1, w1))

                # обратное распространение(back propagation) с использованием градиентного спуска
                layer2_error = y_train - layer2 # производная функции потерь = производная квадратичных потерь 
                layer2_delta = layer2_error * sigmoid_deriv(layer2)
    
                layer1_error = layer2_delta.dot(w1.T)
                layer1_delta = layer1_error * sigmoid_deriv(layer1)
                # коррекция
                w1 += layer1.T.dot(layer2_delta) * n
                w0 += layer0.T.dot(layer1_delta) * n
                # метрика модели
                error = np.mean(np.abs(layer2_error))
                errors.append(error)
                accuracy = (1 - error) * 100
            
            ### Шаг 4. Демонстрация полученных результатов
        
            layer0_t = X_test
            layer1_t = sigmoid(np.dot(layer0_t, w0))
            layer2_t = sigmoid(np.dot(layer1_t, w1))
            layer2_error_t = y_test - layer2_t

            error_t = np.mean(np.abs(layer2_error_t))
            accuracy_t = (1 - error_t) * 100
            accuracies[np.round(accuracy_t,2)] = (neuron_numb, n, epo, np.round(accuracy,2))

In [5]:
from pprint import pprint
pprint(accuracies)

{94.36: (5, 0.05, 3000, 95.58),
 94.64: (10, 0.05, 3000, 95.84),
 94.69: (25, 0.05, 3000, 95.92),
 94.75: (15, 0.05, 3000, 95.93),
 95.8: (5, 0.05, 4000, 96.76),
 95.99: (10, 0.05, 4000, 96.93),
 96.03: (15, 0.05, 4000, 96.97),
 96.06: (25, 0.05, 4000, 97.03),
 96.39: (5, 0.05, 5000, 97.2),
 96.53: (10, 0.05, 5000, 97.33),
 96.56: (15, 0.05, 5000, 97.36),
 96.59: (25, 0.05, 5000, 97.41),
 96.66: (5, 0.1, 3000, 97.25),
 96.74: (10, 0.1, 3000, 97.7),
 96.76: (15, 0.1, 3000, 97.74),
 96.78: (25, 0.1, 3000, 97.77),
 96.92: (5, 0.1, 4000, 97.45),
 96.96: (10, 0.1, 4000, 97.82),
 96.98: (15, 0.1, 4000, 97.85),
 97.0: (25, 0.1, 4000, 97.87),
 97.04: (15, 0.15, 3000, 98.0),
 97.06: (10, 0.15, 3000, 98.01),
 97.08: (5, 0.1, 5000, 97.56),
 97.09: (5, 0.15, 4000, 97.58),
 97.11: (25, 0.15, 3000, 98.06),
 97.12: (15, 0.15, 4000, 98.0),
 97.14: (25, 0.1, 5000, 97.93),
 97.16: (25, 0.15, 4000, 98.05),
 97.17: (5, 0.15, 5000, 97.64),
 97.19: (15, 0.15, 5000, 98.0),
 97.22: (25, 0.15, 5000, 98.04)}


In [6]:
best = max(accuracies)
print('Лучшее качество на тесте: {}\nКоличество нейронов: {}\nСкорость обучения: \
{}\nКоличество эпох: {}\nКачество на обучающей выборке: {}'\
.format(best, accuracies[best][0], accuracies[best][1], accuracies[best][2], accuracies[best][3]))

Лучшее качество на тесте: 97.22
Количество нейронов: 25
Скорость обучения: 0.15
Количество эпох: 5000
Качество на обучающей выборке: 98.04


Начиная с высоких значений epoch и speed происходит быстрое переобучение с высоким результатом на обучающей выборке и низким на тестовой. Плавное же наращивание параметров позволяет следить за переобучением, достигая более высоких результатов.